In [ ]:
# 카카오톡으로 챗GPT 만들기
from flask import Flask, jsonify, request
from openai import OpenAI

app = Flask(__name__)

# 채팅 모델 컨텍스트
CHAT_MODEL_CONTEXT = "당신은 한국어-영어 번역가 입니다. 한국어가 입력되면 그걸 영어로 번역해주세요"

# OpenAI API 클라이언트 설정
client = OpenAI(api_key="이곳에 여러분의 api key를 넣어주세요")


# API 엔드포인트
@app.route('/', methods=['POST'])
def chat_response():
    # 사용자 요청 데이터 추출
    user_request = request.json.get('userRequest', {})
    callback_url = user_request.get('callbackUrl')
    utterance = user_request.get('utterance', '')
    print(f"user_request: {user_request}")
    print(f"callback_url: {callback_url}")
    print(f"utterance: {utterance}")

    # 시스템 컨텍스트로 메시지 리스트 초기화
    messageList = [{"role": "system", "content": CHAT_MODEL_CONTEXT}]

    # 사용자 메시지를 메시지 리스트에 추가
    dicMessage = {"role": "user", "content": utterance}
    messageList.append(dicMessage)

    # OpenAI 채팅 모델에서 응답 받기
    completion = client.chat.completions.create(
        model="gpt-4o",
		max_tokens=200,
        messages=messageList
    )
    response_message = completion.choices[0].message

    # 응답 페이로드 준비
    response = {
        "version": "2.0",
        "template": {
            "outputs": [
                {
                    "simpleText": {
                        "text": f"{response_message.content}"

                    }
                }
            ]
        }
    }

    return jsonify(response)


# 카카오톡으로 챗GPT 애플리케이션 실행
if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080)